In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import time
import requests
import pandas as pd

In [2]:
url = 'https://www.zomato.com/allahabad/nirala-misthan-bhandar-chowk/order'
s = Service("C:/chromedriver-win64/chromedriver.exe")

In [3]:
driver = webdriver.Chrome(service = s)
driver.get(url)
time.sleep(4)

In [4]:
height = driver.execute_script("return document.body.scrollHeight")

In [5]:
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    time.sleep(6)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if height == new_height:
        break
    height = new_height

In [6]:
html = driver.page_source

In [7]:
session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.zomato.com/'
})

response = session.get(url)
print(response.status_code)

200


In [8]:
soup = BeautifulSoup(html, 'html.parser')

In [11]:
image_tags = soup.find_all('img', class_='sc-s1isp7-5 fyZwWD') 

In [12]:
image_tags

[<img alt="Masala Dosa" class="sc-s1isp7-5 fyZwWD" loading="lazy" src="https://b.zmtcdn.com/data/dish_photos/26a/486fd188e5489e2724c3e45d3986726a.png?fit=around|130:130&amp;crop=130:130;*,*"/>,
 <img alt="Special Kaju Paneer Dosa" class="sc-s1isp7-5 fyZwWD" loading="lazy" src="https://b.zmtcdn.com/data/dish_photos/9c1/adf54a7cb346b45b544ec34ebd4499c1.png?fit=around|130:130&amp;crop=130:130;*,*"/>,
 <img alt="Pav Bhaji [1 Plate, 2 Pav]" class="sc-s1isp7-5 fyZwWD" loading="lazy" src="https://b.zmtcdn.com/data/dish_photos/238/f6c649be00d9fc092ae069054b4b3238.png?fit=around|130:130&amp;crop=130:130;*,*"/>,
 <img alt="Paper Plain Dosa" class="sc-s1isp7-5 fyZwWD" loading="lazy" src="https://b.zmtcdn.com/data/dish_photos/62e/bb1d8658b252597378fe388966c0662e.png?fit=around|130:130&amp;crop=130:130;*,*"/>,
 <img alt="Veg Chowmein [1 Plate]" class="sc-s1isp7-5 fyZwWD" loading="lazy" src="https://b.zmtcdn.com/data/dish_photos/fcd/7b3f0185d7da6a242c7f950083975fcd.png?fit=around|130:130&amp;crop=13

In [16]:
import os
output_dir = "menu_images"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [17]:
# Download images and save item names
item_data = []  # To store item names and image paths
for idx, img_tag in enumerate(image_tags, start=1):
    img_url = img_tag.get('src') or img_tag.get('data-src')
    item_name = img_tag.get('alt')  # Extract the item name from 'alt' attribute
    if img_url and item_name:
        try:
            # Sanitize item name to use as a file name
            sanitized_name = "".join(c if c.isalnum() or c in " _-" else "_" for c in item_name)
            img_name = f"{sanitized_name}.jpg"
            img_path = os.path.join(output_dir, img_name)

            # Download and save the image
            img_data = requests.get(img_url).content
            with open(img_path, 'wb') as img_file:
                img_file.write(img_data)

            print(f"Downloaded: {item_name} -> {img_name}")
            item_data.append({'Item Name': item_name, 'Image Path': img_path})
        except Exception as e:
            print(f"Failed to download image for {item_name}: {e}")

Downloaded: Masala Dosa -> Masala Dosa.jpg
Downloaded: Special Kaju Paneer Dosa -> Special Kaju Paneer Dosa.jpg
Downloaded: Pav Bhaji [1 Plate, 2 Pav] -> Pav Bhaji _1 Plate_ 2 Pav_.jpg
Downloaded: Paper Plain Dosa -> Paper Plain Dosa.jpg
Downloaded: Veg Chowmein [1 Plate] -> Veg Chowmein _1 Plate_.jpg
Downloaded: Motichoor Laddu -> Motichoor Laddu.jpg
Downloaded: Special Mewa Motichoor Laddu -> Special Mewa Motichoor Laddu.jpg
Downloaded: Besan Laddu -> Besan Laddu.jpg
Downloaded: Kaju Burfi -> Kaju Burfi.jpg
Downloaded: Sethaura -> Sethaura.jpg
Downloaded: Gond Laddu -> Gond Laddu.jpg
Downloaded: Standard Burfi -> Standard Burfi.jpg
Downloaded: Lal Peda -> Lal Peda.jpg
Downloaded: Karachi Halwa -> Karachi Halwa.jpg
Downloaded: Parval Laata [250 g] -> Parval Laata _250 g_.jpg
Downloaded: Pateesha -> Pateesha.jpg
Downloaded: Plain Peda [Safed Peda] -> Plain Peda _Safed Peda_.jpg
Downloaded: Sutarfeni [250 g] -> Sutarfeni _250 g_.jpg
Downloaded: Gulab Jamun [1 Piece] -> Gulab Jamun _1 Pi

In [19]:
driver.quit()